In [13]:
#1 Load Excel data into a pandas DataFrame
import pandas as pd
import os.path

filename = "FM-0403v8.xlsm"
if not os.path.isfile('FM-0403v8.xlsm'):
    raise ValueError('No')

data = pd.read_excel(filename, sheet_name='administrativeData')
measure = pd.read_excel(filename, sheet_name='measurement')
result = pd.read_excel(filename, sheet_name='result-template')

In [14]:
#2 Convert result to dict
result_dict = {}

print(len(result.columns.tolist()))
print(result.columns.tolist())

#for i in range(len(result.columns.tolist())):

for i in range(len(result.columns.tolist())):
    #if i==3: #i mod 2 = 0 that mean it is an even number
        unit = result.values.tolist()[0][i] #row ที่ 0 และ column ที่ i+1 = เลขคี่
        result_dict[result.columns.tolist()[i]] = [" ".join([str(a[i]) for a in result.values.tolist()])] #list compehension สร้าง list ขึ้นมาจาก for loop โดยที่ for loop อยู่ใน list นั้น
print(result_dict)

9
['RadQ', 'ReferenceValue', 'RangeCalibration', 'UUCMean', 'UUCadjust', 'unit', 'Uncertainty', 'CalibrationFactor', 'unitU']
{'RadQ': ['Cs-137 Cs-137 Cs-137'], 'ReferenceValue': ['1.55 36.0 148.0'], 'RangeCalibration': ['1 10 100'], 'UUCMean': ['1.22 13.4 134.0'], 'UUCadjust': ['1.22 13.4 134.0'], 'unit': ['mSv/h mR/h mR/h'], 'Uncertainty': ['12 8 7'], 'CalibrationFactor': ['1.27 2.69 1.1'], 'unitU': ['&#37; &#37; &#37;']}


In [15]:
#3 Create XML file
import xml.etree.ElementTree as ET
import datetime

# Define XML namespace
nsdcc = {'dcc': 'administrativeData'}
nssi = {'si': 'measurementResults'}

# Create the root element of the XML tree
root = ET.Element('{'+nsdcc['dcc']+'}digitalcalibrationCertificate')
row_element = ET.SubElement(root, '{'+nsdcc['dcc']+'}administrativeData')
row_element_measurement = ET.SubElement(root, '{'+nsdcc['dcc']+'}measurementResults')

# Loop through the dictionary and create XML elements
for row in data.values:
    col_element = ET.SubElement(row_element, '{'+nsdcc['dcc']+'}'+row[0])
    col_element.text = row[1]
    # If the value is an integer, convert it to a string before serializing it
    if isinstance(row[1], (datetime.date,datetime.time)):
        #print(row[1])
        date_ = datetime.datetime.strftime(row[1],"%d %B %Y")
        #print(date_)
        col_element.text = date_

    if isinstance(row[1],int):
        textint = str(row[1])
        col_element.text = textint   

for row in measure.values:
    col_element = ET.SubElement(row_element_measurement, '{'+nsdcc['dcc']+'}'+row[0])
    col_element.text = row[1]

print(result_dict.items())

for key, value in result_dict.items():

    row_result = ET.SubElement(row_element_measurement, '{'+nssi['si']+'}results')

    # Create a name element for the result
    name_element = ET.SubElement(row_result, '{'+nssi['si']+'}'+ 'name')
    name_element.text = key

    # Create a value element for the result
    value_element = ET.SubElement(row_result, '{'+nssi['si']+'}'+ 'value')

    # If the value is a list, iterate over it and add each value to the XML tree
    if isinstance(value, list):
        for value_item in value:
            value_element.text = value_item
    else:
        value_element.text = str(value)

    # Create a unit element for the result
    #unit_element = ET.SubElement(row_result, '{'+nssi['si']+'}'+ 'unit')
    #unit_element.text = str(value)

# Create an ElementTree object with the root element
tree = ET.ElementTree(root)

# Set namespace for the root element//registers a namespace prefix
ET.register_namespace('dcc', nsdcc['dcc'])
#ET.register_namespace('si', nssi['si'])

# Write the XML tree to a file
tree.write('SSDL_OAP_schema.xml', encoding='utf-8', xml_declaration=True)

dict_items([('RadQ', ['Cs-137 Cs-137 Cs-137']), ('ReferenceValue', ['1.55 36.0 148.0']), ('RangeCalibration', ['1 10 100']), ('UUCMean', ['1.22 13.4 134.0']), ('UUCadjust', ['1.22 13.4 134.0']), ('unit', ['mSv/h mR/h mR/h']), ('Uncertainty', ['12 8 7']), ('CalibrationFactor', ['1.27 2.69 1.1']), ('unitU', ['&#37; &#37; &#37;'])])


In [16]:
#4 Modify XML file

#Parse the XML file
tree = ET.parse('SSDL_OAP_schema.xml')

#Find the root element
root = tree.getroot()

#Count the number of <Element {https://ptb.de/si}results>
namespace_si = '{measurementResults}'
count_results = 0
for element in row_element_measurement:
    # Check if the element's tag matches the desired tag
    if element.tag == f"{namespace_si}results":
        count_results += 1
print(count_results)
print(col_element)

#Modify results
for i in range(1,count_results+1):
    # Find the <si:hybrid> element and extract its value
    xpath = ".//{administrativeData}measurementResults/{measurementResults}results["+ str(i)+"]/{measurementResults}value"
    hybrid_element = root.find(xpath, namespaces=nssi)
   
    # Check if the element exists
    if hybrid_element is not None:
        hybrid_value = hybrid_element.text
        # Print the value
        print(hybrid_value)
    else:
        print("Element <si:value> not found in the XML file.")

    # Array_to_list
    data = str(hybrid_value)

    # Assuming you have an array of items
    item_array = data.split(' ')

    path = ".//{administrativeData}measurementResults/{measurementResults}results["+ str(i)+"]"
    child_element = tree.find(path)  # Replace with your desired child element
    child_element.remove(hybrid_element)
    node = root.find(path)


    for item in item_array:
        new_element = ET.Element('{measurementResults}value')
        new_element.text = item

        item_node = root.find(path)
        item_node.append(new_element)
        
# Create an ElementTree object with the root element
tree = ET.ElementTree(root)

# Set namespace for the root element//registers a namespace prefix
ET.register_namespace('dcc', nsdcc['dcc'])
ET.register_namespace('si', nssi['si'])

# Write the XML tree to a file
tree.write('demo-modify-2.xml', encoding='utf-8', xml_declaration=True)

9
<Element '{administrativeData}statementP6' at 0x0000016F7261AC50>
Cs-137 Cs-137 Cs-137
1.55 36.0 148.0
1 10 100
1.22 13.4 134.0
1.22 13.4 134.0
mSv/h mR/h mR/h
12 8 7
1.27 2.69 1.1
&#37; &#37; &#37;


In [20]:
#5 Preview to pdf
import lxml.html
from lxml import etree
xslt_doc = etree.parse('survey_meter_template.xslt')
xslt_transform = etree.XSLT(xslt_doc)

source_doc = etree.parse('demo-modify-2.xml')
output_doc = xslt_transform(source_doc)

print(str(output_doc))
output_doc.write('cert-preview.html', pretty_print = True)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns:dcc="administrativeData" xmlns:si="measurementResults"><link href="https://fonts.googleapis.com/css?family=Sarabun" rel="stylesheet"></html>

